Найдите количество вопросов, которые набрали больше 300 очков или как минимум 100 раз были добавлены в «Закладки».

In [ ]:
'''
SELECT COUNT(id)
FROM stackoverflow.posts 
WHERE post_type_id = 1
  AND (score > 300
       OR favorites_count >=100);
'''


Сколько в среднем в день задавали вопросов с 1 по 18 ноября 2008 включительно? Результат округлите до целого числа.

In [ ]:
'''
SELECT ROUND(AVG(count_id))
FROM (SELECT EXTRACT(DAY FROM creation_date) AS day,
             COUNT(id) AS count_id
      FROM stackoverflow.posts 
      WHERE post_type_id = 1 
          AND DATE_TRUNC('day', creation_date) BETWEEN '2008-11-01' 
             AND '2008-11-18'
      GROUP BY day) AS posts_grouped
'''

Сколько пользователей получили значки сразу в день регистрации? Выведите количество уникальных пользователей.

In [ ]:
'''
SELECT COUNT(DISTINCT u.id)
FROM stackoverflow.users AS u
LEFT JOIN stackoverflow.badges AS b ON u.id = b.user_id
WHERE CAST(u.creation_date AS date) = CAST(b.creation_date AS date);
'''


Сколько уникальных постов пользователя с именем Joel Coehoorn получили хотя бы один голос?

In [ ]:
'''
SELECT COUNT(DISTINCT p.id)
FROM stackoverflow.posts AS p 
JOIN stackoverflow.users u ON p.user_id = u.id
JOIN stackoverflow.votes AS v ON p.id = v.post_id 
WHERE u.display_name = 'Joel Coehoorn';
'''

Выгрузите все поля таблицы vote_types. Добавьте к таблице поле rank, в которое войдут номера записей в обратном порядке. Таблица должна быть отсортирована по полю id.

In [ ]:
'''
SELECT *,
       ROW_NUMBER() OVER (ORDER BY id DESC) as rank
FROM stackoverflow.vote_types 
ORDER BY id;
'''

Отберите 10 пользователей, которые поставили больше всего голосов типа Close. Отобразите таблицу из двух полей: идентификатором пользователя и количеством голосов. Отсортируйте данные сначала по убыванию количества голосов, потом по убыванию значения идентификатора пользователя.

In [ ]:
'''
SELECT u.id,
       COUNT(v.id)
FROM stackoverflow.votes AS v
JOIN stackoverflow.users AS u ON v.user_id = u.id 
JOIN stackoverflow.vote_types AS vt ON v.vote_type_id = vt.id 
WHERE vt.name = 'Close'
GROUP BY u.id
ORDER BY COUNT(v.id) DESC, u.id DESC
LIMIT 10;
'''

Отберите 10 пользователей по количеству значков, полученных в период с 15 ноября по 15 декабря 2008 года включительно.
Отобразите несколько полей:
* идентификатор пользователя;
* число значков;
* место в рейтинге — чем больше значков, тем выше рейтинг.

Пользователям, которые набрали одинаковое количество значков, присвойте одно и то же место в рейтинге.

Отсортируйте записи по количеству значков по убыванию, а затем по возрастанию значения идентификатора пользователя.

In [ ]:
'''
SELECT u.id,
       COUNT(b.id) AS badges_count,
       DENSE_RANK() OVER (ORDER BY COUNT(b.id) DESC)
FROM stackoverflow.users u 
JOIN stackoverflow.badges b ON u.id = b.user_id
WHERE DATE_TRUNC('day', b.creation_date) BETWEEN '2008-11-15' AND '2008-12-15'
GROUP BY u.id 
ORDER BY badges_count DESC, u.id
LIMIT 10;
'''

Сколько в среднем очков получает пост каждого пользователя?
Сформируйте таблицу из следующих полей:
* заголовок поста;
* идентификатор пользователя;
*  очков поста;
* среднее число очков пользователя за пост, округлённое до целого числа.

Не учитывайте посты без заголовка, а также те, что набрали ноль очков.

In [ ]:
'''
SELECT p.title,
       u.id,
       score,
       ROUND(AVG(score) OVER (PARTITION BY u.id))
FROM stackoverflow.posts AS p
LEFT JOIN stackoverflow.users AS u ON p.user_id = u.id  
WHERE score != 0 AND title IS NOT NULL;
'''


Отобразите заголовки постов, которые были написаны пользователями, получившими более 1000 значков. Посты без заголовков не должны попасть в список.

In [ ]:
'''
SELECT title
FROM stackoverflow.posts AS p
LEFT JOIN stackoverflow.users AS uu ON p.user_id = uu.id
WHERE p.user_id IN
    (SELECT u.id
     FROM stackoverflow.users AS u
     JOIN stackoverflow.badges AS b ON u.id = b.user_id
     GROUP BY u.id
     HAVING COUNT(DISTINCT b.id)>1000)
  AND title IS NOT NULL;
'''

Напишите запрос, который выгрузит данные о пользователях из США (англ. United States). Разделите пользователей на три группы в зависимости от количества просмотров их профилей:
* пользователям с числом просмотров больше либо равным 350 присвойте группу 1;
* пользователям с числом просмотров меньше 350, но больше либо равно 100 — группу 2;
* пользователям с числом просмотров меньше 100 — группу 3.

Отобразите в итоговой таблице идентификатор пользователя, количество просмотров профиля и группу. Пользователи с нулевым количеством просмотров не должны войти в итоговую таблицу.

In [ ]:
'''
SELECT id,
       views,
       CASE 
           WHEN views >= 350 THEN 1
           WHEN views >= 100 THEN 2
           ELSE 3
       END 
FROM stackoverflow.users AS u 
WHERE location LIKE '%United States%' AND views > 0
'''

Дополните предыдущий запрос. Отобразите лидеров каждой группы — пользователей, которые набрали максимальное число просмотров в своей группе. Выведите поля с идентификатором пользователя, группой и количеством просмотров. Отсортируйте таблицу по убыванию просмотров, а затем по возрастанию значения идентификатора.

In [ ]:
'''
SELECT id,
       q1,
       views
FROM (SELECT *,
             MAX(views) OVER (PARTITION BY q1) AS val_max
      FROM (SELECT id,
                   views,
                   CASE 
                       WHEN views >= 350 THEN 1
                       WHEN views >= 100 THEN 2
                       ELSE 3
                   END AS q1
            FROM stackoverflow.users AS u 
            WHERE location LIKE '%United States%' AND views> 0) AS a) AS q2
WHERE val_max = views
ORDER BY views DESC, id
'''

Посчитайте ежедневный прирост новых пользователей в ноябре 2008 года. Сформируйте таблицу с полями:
*  номер дня;
* число пользователей, зарегистрированных в этот день;
* сумму пользователей с накоплением.

In [ ]:
'''
WITH q1 AS (SELECT EXTRACT(DAY FROM creation_date) as reg,
                        COUNT(id) AS q1_id
                 FROM stackoverflow.users u 
                 WHERE EXTRACT(MONTH FROM creation_date) = 11
                           AND EXTRACT(YEAR FROM creation_date) = 2008
                 GROUP BY reg)
SELECT *,
       SUM(q1_id) OVER (ORDER BY reg)
FROM q1;
'''

Для каждого пользователя, который написал хотя бы один пост, найдите интервал между регистрацией и временем создания первого поста. Отобразите:
* идентификатор пользователя;
* разницу во времени между регистрацией и первым постом.

In [ ]:
'''
WITH q1 AS (SELECT user_id,
                    p.creation_date AS p_cd,
                    ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY p.creation_date) AS cd_t,
                    u.creation_date AS cd 
             FROM stackoverflow.posts AS p JOIN stackoverflow.users AS u ON p.user_id = u.id 
             ORDER BY user_id)
SELECT user_id,
       p_cd - cd
FROM q1
WHERE cd_t = 1
'''

Выведите общую сумму просмотров постов за каждый месяц 2008 года. Если данных за какой-либо месяц в базе нет, такой месяц можно пропустить. Результат отсортируйте по убыванию общего количества просмотров.

In [ ]:
'''
SELECT DATE_TRUNC('month', creation_date)::date AS month_date,
       SUM(views_count) AS total_views
FROM stackoverflow.posts 
GROUP BY month_date
ORDER BY total_views DESC;
'''

Выведите имена самых активных пользователей, которые в первый месяц после регистрации (включая день регистрации) дали больше 100 ответов. Вопросы, которые задавали пользователи, не учитывайте. Для каждого имени пользователя выведите количество уникальных значений user_id. Отсортируйте результат по полю с именами в лексикографическом порядке.

In [ ]:
'''
SELECT u.display_name,
       COUNT(DISTINCT user_id)
FROM stackoverflow.posts p
JOIN stackoverflow.post_types pt ON p.post_type_id = pt.id
JOIN stackoverflow.users u ON u.id = p.user_id
WHERE DATE_TRUNC('day', p.creation_date) >= DATE_TRUNC('day', u.creation_date)
  AND DATE_TRUNC('day', p.creation_date) <= DATE_TRUNC('day', u.creation_date) + INTERVAL '1 month'
  AND pt.type = 'Answer'
GROUP BY u.display_name
HAVING COUNT(*) > 100
ORDER BY display_name;
'''

Выведите количество постов за 2008 год по месяцам. Отберите посты от пользователей, которые зарегистрировались в сентябре 2008 года и сделали хотя бы один пост в декабре того же года. Отсортируйте таблицу по значению месяца по убыванию.

In [ ]:
'''
SELECT DATE_TRUNC('month', creation_date)::date AS monthly,
       COUNT(*)
FROM stackoverflow.posts
WHERE user_id IN
    (SELECT DISTINCT u.id
     FROM stackoverflow.users u
     JOIN stackoverflow.posts p ON u.id = p.user_id
     WHERE DATE_TRUNC('month', u.creation_date) = '2008-09-01'
       AND DATE_TRUNC('month', p.creation_date) = '2008-12-01')
GROUP BY monthly
ORDER BY monthly DESC;
'''


Используя данные о постах, выведите несколько полей:
* идентификатор пользователя, который написал пост;
* дата создания поста;
* количество просмотров у текущего поста;
* сумму просмотров постов автора с накоплением.

Данные в таблице должны быть отсортированы по возрастанию идентификаторов пользователей, а данные об одном и том же пользователе — по возрастанию даты создания поста.

In [ ]:
'''
SELECT user_id,
       creation_date,
       views_count,
       SUM(views_count) OVER (PARTITION BY user_id ORDER BY creation_date) AS vieas
FROM stackoverflow.posts;
'''


На сколько процентов менялось количество постов ежемесячно с 1 сентября по 31 декабря 2008 года? Отобразите таблицу со следующими полями:
* номер месяца;
* количество постов за месяц;
* процент, который показывает, насколько изменилось количество постов в текущем месяце по сравнению с предыдущим.

Если постов стало меньше, значение процента должно быть отрицательным, если больше — положительным. Округлите значение процента до двух знаков после запятой.

Напомним, что при делении одного целого числа на другое в PostgreSQL в результате получится целое число, округлённое до ближайшего целого вниз. Чтобы этого избежать, переведите делимое в тип numeric.

In [ ]:
'''
SELECT *,
       ROUND((100 * CAST(posts_count AS numeric) / LAG(posts_count, 1) OVER (ORDER BY posts_count DESC) - 100),2) AS percentage   
FROM (SELECT EXTRACT(MONTH FROM creation_date) AS creation_monthly,
             COUNT(id) AS posts_count
      FROM stackoverflow.posts 
      WHERE creation_date BETWEEN '2008-09-01' AND '2008-12-31'
      GROUP BY creation_monthly
      ORDER BY creation_monthly) AS foo;
'''


Выгрузите данные активности пользователя, который опубликовал больше всего постов за всё время. Выведите данные за октябрь 2008 года в таком виде:
* номер недели;
* дата и время последнего поста, опубликованного на этой неделе.

In [ ]:
'''
SELECT week_creation,
       creation_date
FROM
  (SELECT *,
          ROW_NUMBER() OVER (PARTITION BY week_creation
                             ORDER BY creation_date DESC) AS number_posts
   FROM
     (SELECT id,
             title,
             user_id,
             EXTRACT(WEEK
                     FROM creation_date) AS week_creation,
             creation_date
      FROM stackoverflow.posts
      WHERE EXTRACT(MONTH
                    FROM creation_date) = 10
        AND user_id IN
          (SELECT user_id
           FROM stackoverflow.posts
           GROUP BY user_id
           ORDER BY COUNT(user_id) DESC
           LIMIT 1)) AS p_1) AS p_2
WHERE number_posts = 1;
'''